In [1]:
import simulate as sim
import numpy as np

# Timing of the functions involved in a Verlet step

We test and compare the performance of the functions involved in a Verlet step in order to know which one is the bottleneck of the simulation. 

The two functions that consume more time are 
1. `sim.lj_force` $\approx$ 4.3 ms (for $N=200$)
2. `sim.atomic_distances` $\approx$ 3.5 ms (for $N=200$)

while the rest are execture in less than 7 µs (for $N=200$).

The total time of a Verlet step is: 16.5 ms ± 523 µs (for $N=200$)

In [2]:
# Input parameters
N = 200 # number of particles
d = 3 # dimensionality of the box
L = 3 * N**(1/d) # box length in units of sigma
T = 300 # temperature in SI
timestep = 1E-4

pos = L*np.random.rand(N, d)
vel = np.sqrt(sim.KB*T/sim.EPSILON) - 2*np.sqrt(sim.KB*T/sim.EPSILON)*np.random.rand(N, d)

In [4]:
print("atomic distances")
rel_pos, rel_dist = sim.atomic_distances(pos, L)
%timeit sim.atomic_distances(pos, L)

atomic distances
3.57 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
print("rel_dis np.linalg.norm")
%timeit np.linalg.norm(rel_pos, axis=2)
print("rel_dis np.eigsum")
%timeit np.sqrt(np.einsum('ijk, ijk->ij', rel_pos, rel_pos, optimize="optimal"))

rel_dis np.linalg.norm
1.18 ms ± 90.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
rel_dis np.eigsum
562 µs ± 65.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
print("lj_force")
total_force = sim.lj_force(rel_pos, rel_dist)
%timeit sim.lj_force(rel_pos, rel_dist)

lj_force
4.38 ms ± 825 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
print("update position")
pos = pos + vel*timestep + 0.5*timestep**2*total_force
%timeit pos + vel*timestep + 0.5*timestep**2*total_force

update position
5.09 µs ± 526 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
print("atomic_distances")
rel_pos, rel_dist = sim.atomic_distances(pos, L)
%timeit sim.atomic_distances(pos, L)

atomic_distances
4.06 ms ± 771 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
print("lj_force")
total_force_next = sim.lj_force(rel_pos, rel_dist)
%timeit sim.lj_force(rel_pos, rel_dist)

lj_force
4.31 ms ± 693 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
print("update velocity")
vel = vel + 0.5*(total_force + total_force_next)*timestep
%timeit vel + 0.5*(total_force + total_force_next)*timestep

update velocity
4.73 µs ± 417 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
print("update periodic BC")
pos = pos - np.floor(pos/L)*L
%timeit pos - np.floor(pos/L)*L

update periodic BC
6.02 µs ± 554 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
print("TOTAL SIMULATION OF ONE STEP")
%timeit sim.simulate_step_verlet(pos, vel, timestep, L)

TOTAL SIMULATION OF ONE STEP
16.5 ms ± 523 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
